In [46]:
import pandas
import os

In [47]:
traindf=pandas.read_csv(os.path.join('res', 'titanic', 'train.csv'))

testdf=pandas.merge(pandas.read_csv(os.path.join('res', 'titanic', 'test.csv')),
                    pandas.read_csv(os.path.join('res', 'titanic', 'gender_submission.csv')),
                    on="PassengerId")

In [48]:
for k in traindf.keys():
    print('{0}: {1}'.format(k, len(traindf[k].unique())))

PassengerId: 891
Survived: 2
Pclass: 3
Name: 891
Sex: 2
Age: 89
SibSp: 7
Parch: 7
Ticket: 681
Fare: 248
Cabin: 148
Embarked: 4


In [49]:
for k in traindf.keys():
    if len(traindf[k].unique())<=15:
        print(k)

Survived
Pclass
Sex
SibSp
Parch
Embarked


In [50]:
import math

def forAge(row):
    if row['Age'] < 10:
        return '<10'
    elif math.isnan(row['Age']):
        return "nan"
    else:
        dec = str(int(row['Age']/10))
        return "{0}0's".format(dec)

decade=traindf.apply(forAge, axis=1)
print("decade: {1}".format(k, len(decade.unique())))

decade: 10


In [51]:
import math
def pretreat(df):
    # create a df with discrete variables (len<10)
    lvd=[k for k in df.keys() if len(df[k].unique())<=15]
    res=df[lvd].copy()
    
    forAge = lambda row: "{0}".format(int(row['Age']/10)) if not math.isnan(row['Age']) else str("nan")
    res['decade']=df.apply(forAge, axis=1)
   
    return res

In [52]:
ptraindf=pretreat(traindf)
ptestdf=pretreat(testdf)

In [53]:
ptraindf.to_csv(os.path.join('res', 'titanic', 'post_train.csv'), index=False)
ptestdf.to_csv(os.path.join('res', 'titanic', 'post_test.csv'), index=False)

In [54]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

In [55]:
df = pandas.read_csv(os.path.join('res', 'titanic', 'post_train.csv'))
for k in df.keys():
    print("{} : {}".format(k, df[k].unique()))

Survived : [0 1]
Pclass : [3 1 2]
Sex : ['male' 'female']
SibSp : [1 0 3 4 2 5 8]
Parch : [0 1 2 5 3 4 6]
Embarked : ['S' 'C' 'Q' nan]
decade : ['20th' '30th' nan '50th' '00th' '10th' '40th' '60th' '70th' '80th']


In [91]:
template=gum.BayesNet()
template.add(gum.LabelizedVariable("Survived", "Survived",1))
template.add(gum.LabelizedVariable("Pclass", "Pclass",3))
template.add(gum.LabelizedVariable("Sex", "Sex",0).addLabel("female").addLabel("male"))
template.add(gum.LabelizedVariable("SibSp", "SibSp",8))
template.add(gum.LabelizedVariable("Parch", "Parch",9))
template.add(gum.LabelizedVariable("Embarked", "Embarked",0).addLabel('').addLabel('C').addLabel('Q').addLabel('S'))
#template.add(gum.LabelizedVariable("Decade", "Calculated decade", 10).addLabel(''))
#template.add(gum.LabelizedVariable("decade", "Calculated decade", 0).addLabel('').addLabel('00th').addLabel('10th').addLabel('20th').addLabel('30th').addLabel('40th').addLabel('50th').addLabel('60th').addLabel('70th').addLabel('80th'))
gnb.showBN(template)

In [92]:
learner = gum.BNLearner(os.path.join('res', 'titanic', 'post_train.csv'), template)
bn = learner.learnBN()
bn

UnknownLabelInDatabase: [pyAgrum] Unknown label found in database: The translation could not be found

TypeError: in method 'RangeVariable_index', argument 2 of type 'std::string const &'